# Time Decay
**Truth Premise: When a day ends and a new day begins in life of options premium, its value depreciates because of time decay because it gets closer to expiry and the probability of its contract going as expected by the buyer gets lesser.**

## Strategy
1. Short both put and call at day closing right before the expiry
2. Square of in the morning

## Pseudocode
```python
nifty_at_0320
at_the_money_call_strike
at_the_money_put_strike
call_strike_premium_at_0320
put_strike_premium_at_0320
call_strike_premium_at_0915
put_strike_premium_at_0915
call_strike_premium_at_0916
put_strike_premium_at_0916
call_strike_premium_at_0917
put_strike_premium_at_0917
call_strike_premium_at_0918
put_strike_premium_at_0918
pnl_0915
pnl_0916
pnl_0917
pnl_0918
```

In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *


TEST_START = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
DAILY_INVESTMENT = 1 * (10 ** 5) # 1 lakh
LOT_SIZE = 50

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)
# train_dates = test_dates

@cache
def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

@cache
def get_symbol_first_candle(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open, data.iloc[0].high, data.iloc[0].low, data.iloc[0].close

@cache
def get_first_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].close

@cache
def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

train_dates["previous_trading_day"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, 1), axis=1)

def get_nifty_price(d, t):
    data = ut.get_data(symbol=SYMBOL, date=d, interval=INTERVAL, exchange=EXCHANGE)
    try:
        return data.loc[data.index.time == t].iloc[0].open
    except IndexError:
        return pd.NA
    except AttributeError as e:
        return pd.NA

buy_time = dt.time(hour=15, minute=28)
sell_time = dt.time(hour=9, minute=18)

train_dates = train_dates.copy()

# train_dates["trade_day_before_expiry"] = train_dates.expiry - pd.Timedelta(days=day_before_expiry)
train_dates.loc[:, "nifty_at_0320"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), dt.time(hour=15, minute=20)), axis=1)
train_dates.loc[:, "nifty_at_buy"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), buy_time), axis=1)
train_dates.loc[:, "nifty_at_sell"] = train_dates.apply(lambda r: get_nifty_price(r.name.date(), sell_time), axis=1)
train_dates.loc[:, "nifty_diff"] = train_dates.nifty_at_sell - train_dates.nifty_at_buy

train_dates = train_dates.loc[train_dates.nifty_at_0320.notna()]
train_dates["atm_strike"] = train_dates.apply(lambda trade: round(trade.nifty_at_0320 / 50) * 50, axis=1)

In [2]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if type(pr) == type(pd.NA) or pr.shape[0] == 0:
        return pd.NA
    x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    if x.shape[0] == 0:
        # print(tm)
        # print(td)
        # print(pr.loc[(pr.index.date == td)])
        # print(trade)
        return pd.NA
    return x.iloc[0].close

def get_next_1000(strike, i):
    divider = 100
    reminder = strike % divider
    if reminder != 0:
        return divider * (strike // divider) - i * divider
    return divider * (strike // divider) - (i+1) * divider

last_strike = None
"""
Results Training dataset 2nd expiry
    Call PnL at 9:18 for strike: -10, Total: -759835.0, Per Day: -2140.3802816901407, Days: 355, Pc Mean: -2.14975530819984
Call PnL at 9:18 for strike: -9, Total: -312092.4999999999, Per Day: -857.3969780219777, Days: 364, Pc Mean: -0.8748627952745024
Call PnL at 9:18 for strike: -8, Total: 71502.49999999994, Per Day: 195.36202185792334, Days: 366, Pc Mean: 0.18297912502349303
Call PnL at 9:18 for strike: -7, Total: 506540.0, Per Day: 1380.2179836512262, Days: 367, Pc Mean: 1.369693395280383
Call PnL at 9:18 for strike: -6, Total: 641875.0, Per Day: 1744.2255434782608, Days: 368, Pc Mean: 1.743611133234208
Call PnL at 9:18 for strike: -5, Total: 755737.5, Per Day: 2053.634510869565, Days: 368, Pc Mean: 2.054257620026519
    Call PnL at 9:18 for strike: -4, Total: 826650.0, Per Day: 2246.3315217391305, Days: 368, Pc Mean: 2.2097529699576683
Call PnL at 9:18 for strike: -3, Total: 807337.5000000002, Per Day: 2193.8519021739135, Days: 368, Pc Mean: 2.1964092790268843
Call PnL at 9:18 for strike: -2, Total: 720057.5, Per Day: 1956.6779891304348, Days: 368, Pc Mean: 1.9887079193200083
Call PnL at 9:18 for strike: -1, Total: 641890.0000000001, Per Day: 1744.2663043478265, Days: 368, Pc Mean: 1.8311249840868105
Call PnL at 9:18 for strike: 0, Total: 577307.5000000002, Per Day: 1568.7703804347832, Days: 368, Pc Mean: 1.619612769168954
Call PnL at 9:18 for strike: 1, Total: 526917.5, Per Day: 1455.5732044198894, Days: 362, Pc Mean: 1.5759072833721137
Call PnL at 9:18 for strike: 2, Total: 480467.50000000006, Per Day: 1392.6594202898552, Days: 345, Pc Mean: 1.5418813730866745
Call PnL at 9:18 for strike: 3, Total: 411579.99999999994, Per Day: 1331.9741100323622, Days: 309, Pc Mean: 1.5533763343764866
Call PnL at 9:18 for strike: 4, Total: 276877.5, Per Day: 1036.9943820224719, Days: 267, Pc Mean: 1.2849965646637946
Call PnL at 9:18 for strike: 5, Total: 149872.5000000001, Per Day: 724.0217391304352, Days: 207, Pc Mean: 0.7762541829311271
Call PnL at 9:18 for strike: 6, Total: 154740.0, Per Day: 979.367088607595, Days: 158, Pc Mean: 1.2739409931218943
Call PnL at 9:18 for strike: 7, Total: 10914.999999999995, Per Day: 94.09482758620685, Days: 116, Pc Mean: 0.12636272094800613
Call PnL at 9:18 for strike: 8, Total: 9917.499999999989, Per Day: 113.9942528735631, Days: 87, Pc Mean: 0.18813370356162376
Call PnL at 9:18 for strike: 9, Total: 79105.00000000006, Per Day: 1180.6716417910457, Days: 67, Pc Mean: 1.5818898689081564
Call PnL at 9:18 for strike: 10, Total: 35062.50000000002, Per Day: 637.5000000000003, Days: 55, Pc Mean: 1.1953436107573956
Call PnL at 9:18 for strike: 11, Total: 20739.99999999998, Per Day: 460.88888888888846, Days: 45, Pc Mean: 0.7593888029924587
Call PnL at 9:18 for strike: 12, Total: -55417.49999999998, Per Day: -1288.7790697674413, Days: 43, Pc Mean: -2.0308454534870015
Call PnL at 9:18 for strike: 13, Total: 21262.50000000002, Per Day: 574.6621621621626, Days: 37, Pc Mean: 0.8487115283167249
Call PnL at 9:18 for strike: 14, Total: -24174.999999999978, Per Day: -1510.9374999999986, Days: 16, Pc Mean: -1.9692430667294567

Results Training dataset 1st expiry
Call PnL at 9:18 for strike: -10, Total: -1200977.5, Per Day: -3263.5258152173915, Days: 368, Pc Mean: -3.2645265341374787
Call PnL at 9:18 for strike: -9, Total: -1498797.5, Per Day: -4072.819293478261, Days: 368, Pc Mean: -4.07595378510088
Call PnL at 9:18 for strike: -8, Total: -1580342.5, Per Day: -4294.408967391304, Days: 368, Pc Mean: -4.300811979583259
Call PnL at 9:18 for strike: -7, Total: -1589612.5, Per Day: -4319.599184782609, Days: 368, Pc Mean: -4.33081776982772
Call PnL at 9:18 for strike: -6, Total: -1313277.5, Per Day: -3568.688858695652, Days: 368, Pc Mean: -3.581478936364814
Call PnL at 9:18 for strike: -5, Total: -957462.4999999998, Per Day: -2601.8002717391296, Days: 368, Pc Mean: -2.6178756181139833
Call PnL at 9:18 for strike: -4, Total: -376049.9999999999, Per Day: -1021.8749999999997, Days: 368, Pc Mean: -1.0172680452979237
Call PnL at 9:18 for strike: -3, Total: 861522.5, Per Day: 2341.09375, Days: 368, Pc Mean: 2.3639857872910177
Call PnL at 9:18 for strike: -2, Total: 1831165.0, Per Day: 4975.991847826087, Days: 368, Pc Mean: 5.016709407810276
Call PnL at 9:18 for strike: -1, Total: 1928680.0, Per Day: 5240.978260869565, Days: 368, Pc Mean: 5.2977612436011565
Call PnL at 9:18 for strike: 0, Total: 1511580.0, Per Day: 4107.554347826087, Days: 368, Pc Mean: 4.227216010633293
Call PnL at 9:18 for strike: 1, Total: 1021664.9999999998, Per Day: 2776.263586956521, Days: 368, Pc Mean: 3.0202142490892543
Call PnL at 9:18 for strike: 2, Total: 790220.0000000002, Per Day: 2147.33695652174, Days: 368, Pc Mean: 2.2219541615482883
Call PnL at 9:18 for strike: 3, Total: 529924.9999999999, Per Day: 1443.9373297002721, Days: 367, Pc Mean: 1.659199832293209
Call PnL at 9:18 for strike: 4, Total: 404357.50000000006, Per Day: 1117.0096685082874, Days: 362, Pc Mean: 1.2296016828922414
"""

# for i in range(-15, 30):
# for i in range(5, 10):
# for i in range(10, 15):
# for i in range(-10, -5):
# for i in range(-5, 0):
for i in range(0, 5):
    strike_key = f"cur_call_atm_strike_{i}"
    train_dates[strike_key] = train_dates["atm_strike"].apply(lambda r: get_next_1000(r, i))
    buy_at = f"call_at_0328_{i}"
    sell_at = f"call_at_0918_{i}"
    quantity_key = f"quantity_{i}"
    train_dates[buy_at] = train_dates.apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.previous_trading_day.date(), OPTION_TYPE_CALL, dt.time(hour=15, minute=28)), axis=1)
    train_dates[sell_at] = train_dates.loc[train_dates[buy_at].notna()].apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.name.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=18)), axis=1)
    train_dates[quantity_key] = train_dates.loc[train_dates[buy_at].notna()][buy_at].apply(lambda buy_price: ut.get_quantity(buy_price=buy_price, lot_size=LOT_SIZE, investment=DAILY_INVESTMENT))
    pnl_key = f"call_pnl_0918_{i}"
    train_dates[pnl_key] = train_dates.loc[train_dates[buy_at].notna()][quantity_key] * (train_dates.loc[train_dates[buy_at].notna()][sell_at] - train_dates.loc[train_dates[buy_at].notna()][buy_at])
    pnl_pc_key = f"call_pnl_pc_{i}"
    train_dates[pnl_pc_key] = train_dates.loc[train_dates[buy_at].notna()][pnl_key] * 100 / (train_dates.loc[train_dates[buy_at].notna()][quantity_key] * train_dates.loc[train_dates[buy_at].notna()][buy_at])
    print(f"Call PnL at 9:18 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}, Pc Mean: {train_dates[pnl_pc_key].mean()}")


Call PnL at 9:18 for strike: 0, Total: 1511580.0, Per Day: 4107.554347826087, Days: 368, Pc Mean: 4.227216010633293
Call PnL at 9:18 for strike: 1, Total: 1021664.9999999998, Per Day: 2776.263586956521, Days: 368, Pc Mean: 3.0202142490892543
Call PnL at 9:18 for strike: 2, Total: 790220.0000000002, Per Day: 2147.33695652174, Days: 368, Pc Mean: 2.2219541615482883
Call PnL at 9:18 for strike: 3, Total: 529924.9999999999, Per Day: 1443.9373297002721, Days: 367, Pc Mean: 1.659199832293209
Call PnL at 9:18 for strike: 4, Total: 404357.50000000006, Per Day: 1117.0096685082874, Days: 362, Pc Mean: 1.2296016828922414


In [11]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)
train_dates["dd"] = train_dates["call_pnl_0918_0"]

for idx, row in train_dates.iterrows():
    pr = train_dates.shift(1).loc[idx]
    if pd.isna(pr.atm_strike) or pd.isnull(pr.atm_strike):
        continue
    train_dates.loc[idx, "dd"] = pr.dd + row.call_pnl_0918_0
    # row["dd"] = pr.dd + row.call_pnl_0918_0

def format_float(value):
    return f'{value:.2f}'

# Set the display.float_format option to use the formatting function
pd.options.display.float_format = format_float

num = 0
train_dates[["previous_trading_day", "expiry", "nifty_at_0320", "atm_strike", f"cur_call_atm_strike_{num}", f"call_at_0328_{num}", f"call_at_0918_{num}", f"call_pnl_0918_{num}", "nifty_diff", "nifty_at_buy", "nifty_at_sell", f"call_pnl_pc_{num}", f"quantity_{num}", "dd"]]

,previous_trading_day,expiry,nifty_at_0320,atm_strike,cur_call_atm_strike_0,call_at_0328_0,call_at_0918_0,call_pnl_0918_0,nifty_diff,nifty_at_buy,nifty_at_sell,call_pnl_pc_0,quantity_0,dd
trade_date,,,,,,,,,,,,,,
2021-01-01,2020-12-31,2021-01-07,13979.90,14000,13900,182.00,192.40,5200.00,33.45,13980.20,14013.65,5.71,500.00,5200.00
2021-01-05,2021-01-04,2021-01-07,14129.55,14150,14100,109.90,82.25,-24885.00,-52.30,14143.75,14091.45,-25.16,900.00,-19685.00
2021-01-11,2021-01-08,2021-01-14,14352.45,14350,14300,136.35,167.95,22120.00,55.70,14354.90,14410.60,23.18,700.00,2435.00
2021-01-12,2021-01-11,2021-01-14,14483.75,14500,14400,140.45,121.35,-13370.00,-31.45,14489.30,14457.85,-13.60,700.00,-10935.00
2021-01-14,2021-01-13,2021-01-14,14566.60,14550,14500,85.55,73.00,-14432.50,-18.10,14556.70,14538.60,-14.67,1150.00,-25367.50
2021-01-15,2021-01-14,2021-01-21,14593.15,14600,14500,208.00,198.55,-4252.50,11.00,14596.85,14607.85,-4.54,450.00,-29620.00
2021-01-18,2021-01-15,2021-01-21,14441.85,14450,14400,172.10,140.00,-17655.00,-41.65,14437.30,14395.65,-18.65,550.00,-47275.00
2021-01-22,2021-01-21,2021-01-28,14621.60,14600,14500,229.30,181.85,-18980.00,-69.55,14622.40,14552.85,-20.69,400.00,-66255.00
2021-01-28,2021-01-27,2021-01-28,13971.30,13950,13900,118.00,31.65,-69080.00,-126.65,13964.95,13838.30,-73.18,800.00,-135335.00


In [13]:
"""
Training dataset 1st expiry
-261950.0
2021-03-31 00:00:00
"""

print(train_dates["dd"].min())
print(train_dates["dd"].idxmin())

-261950.0
2021-03-31 00:00:00


In [7]:
train_dates[["trade_day_before_expiry", "atm_strike", "nifty_at_0320", "put_at_0320_250", "put_at_0918_250", "put_pnl_0918_250"]]
print(train_dates[train_dates.put_pnl_0918_250 > 0].shape)
print(train_dates[train_dates.put_pnl_0918_250 <= 0].shape)
train_dates.shape

# train_dates

(39, 81)
(29, 81)


(68, 81)

In [5]:
train_dates[["trade_day_before_expiry", "atm_strike", "nifty_at_0320", "put_at_0320_250", "put_at_0918_250", "put_pnl_0918_250"]]
# train_dates.to_csv("time_decay_results.csv")

,trade_day_before_expiry,atm_strike,nifty_at_0320,put_at_0320_250,put_at_0918_250,put_pnl_0918_250
trade_date,,,,,,
2021-02-03,2021-02-03,14800,14787.50,271.65,281.80,-10.15
2021-03-03,2021-03-03,15250,15259.00,244.65,457.80,-213.15
2021-03-09,2021-03-09,15100,15120.05,241.50,153.05,88.45
2021-03-17,2021-03-17,14700,14714.35,227.35,118.30,109.05
2021-03-24,2021-03-24,14550,14544.80,254.35,320.00,-65.65
2021-03-31,2021-03-31,14700,14686.90,256.85,151.80,105.05
2021-04-28,2021-04-28,14850,14859.20,255.80,102.10,153.70
2021-05-05,2021-05-05,14600,14620.85,220.25,194.95,25.30
2021-05-11,2021-05-11,14850,14850.40,254.20,296.00,-41.80


In [14]:
train_dates.loc[train_dates.pnl_0915 < 0, ["expiry", "atm_strike", "call_at_0320", "call_at_0915", "put_at_0320", "put_at_0915", "call_pnl_0915", "put_pnl_0915", "pnl_0918"]]

,expiry,atm_strike,call_at_0320,call_at_0915,put_at_0320,put_at_0915,call_pnl_0915,put_pnl_0915,pnl_0918
trade_date,,,,,,,,,
2021-01-27,2021-01-28,13950,155.40,63.20,228.05,333.75,92.20,-105.70,-36.45
2021-06-16,2021-06-17,15750,130.40,68.40,247.15,314.35,62.00,-67.20,-6.20
2021-07-07,2021-07-08,15900,90.15,80.15,271.45,292.05,10.00,-20.60,-2.25
2021-10-27,2021-10-28,18200,144.30,99.20,236.50,294.30,45.10,-57.80,-14.40
2021-12-01,2021-12-02,17200,126.40,101.60,263.15,291.60,24.80,-28.45,2.40
2021-12-29,2021-12-30,17200,133.95,93.20,240.05,281.70,40.75,-41.65,4.20
2022-03-09,2022-03-10,16350,183.45,390.70,267.60,64.80,-207.25,202.80,-10.45
2022-05-18,2022-05-19,16250,129.10,13.25,271.05,527.80,115.85,-256.75,-181.80
2022-06-08,2022-06-09,16350,129.40,63.20,257.50,333.95,66.20,-76.45,-7.65


In [5]:
train_dates = pd.read_csv("time_decay_results.csv")

,trade_date,previous_trading_day,expiry,trade_day_before_expiry,nifty_at_0320,atm_strike,cur_put_atm_strike,put_at_0320_-500,put_at_0918_-500,put_pnl_0918_-500,...,put_pnl_0918_550,put_at_0320_600,put_at_0918_600,put_pnl_0918_600,put_at_0320_650,put_at_0918_650,put_pnl_0918_650,put_at_0320_700,put_at_0918_700,put_pnl_0918_700
0,2021-02-03,2021-02-02,2021-02-04,2021-02-03,14787.50,14800,15500,5.35,1.85,3.50,...,-32.95,615.85,654.40,-38.55,670.00,704.85,-34.85,717.50,735.45,-17.95
1,2021-03-03,2021-03-02,2021-03-04,2021-03-03,15259.00,15250,15950,5.65,10.05,-4.40,...,-194.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-09,2021-03-08,2021-03-10,2021-03-09,15120.05,15100,15800,2.90,0.95,1.95,...,79.95,594.45,501.90,92.55,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-03-17,2021-03-16,2021-03-18,2021-03-17,14714.35,14700,15400,1.55,0.35,1.20,...,116.35,565.75,448.70,117.05,624.15,499.75,124.40,662.95,547.70,115.25
4,2021-03-24,2021-03-23,2021-03-25,2021-03-24,14544.80,14550,15250,3.00,2.35,0.65,...,-68.70,599.15,682.00,-82.85,644.35,718.30,-73.95,694.70,780.20,-85.50
5,2021-03-31,2021-03-30,2021-04-01,2021-03-31,14686.90,14700,15400,3.25,1.05,2.20,...,63.65,602.95,494.00,108.95,652.85,585.15,67.70,701.00,613.05,87.95
6,2021-04-28,2021-04-27,2021-04-29,2021-04-28,14859.20,14850,15550,6.00,1.70,4.30,...,164.10,NaN,NaN,NaN,649.00,488.60,160.40,NaN,NaN,NaN
7,2021-05-05,2021-05-04,2021-05-06,2021-05-05,14620.85,14600,15300,2.25,0.70,1.55,...,17.90,566.00,550.15,15.85,617.30,561.95,55.35,665.00,641.40,23.60
8,2021-05-11,2021-05-10,2021-05-12,2021-05-11,14850.40,14850,15550,1.90,0.75,1.15,...,-64.45,NaN,NaN,NaN,651.00,717.20,-66.20,NaN,NaN,NaN
9,2021-06-02,2021-06-01,2021-06-03,2021-06-02,15587.45,15600,16300,1.80,0.75,1.05,...,NaN,NaN,NaN,NaN,661.35,599.15,62.20,NaN,NaN,NaN


In [10]:
train_dates["rolling_sum_7"] = train_dates["put_pnl_0918_250"].rolling(window=1).sum()
train_dates["rolling_sum_10"] = train_dates["put_pnl_0918_250"].rolling(window=10).sum()

In [11]:
train_dates[["trade_day_before_expiry", "atm_strike", "nifty_at_0320", "put_at_0320_250", "put_at_0918_250", "put_pnl_0918_250", "rolling_sum_7", "rolling_sum_10"]]

,trade_day_before_expiry,atm_strike,nifty_at_0320,put_at_0320_250,put_at_0918_250,put_pnl_0918_250,rolling_sum_7,rolling_sum_10
0,2021-02-03,14800,14787.50,271.65,281.80,-10.15,NaN,NaN
1,2021-03-03,15250,15259.00,244.65,457.80,-213.15,NaN,NaN
2,2021-03-09,15100,15120.05,241.50,153.05,88.45,NaN,NaN
3,2021-03-17,14700,14714.35,227.35,118.30,109.05,NaN,NaN
4,2021-03-24,14550,14544.80,254.35,320.00,-65.65,NaN,NaN
5,2021-03-31,14700,14686.90,256.85,151.80,105.05,NaN,NaN
6,2021-04-28,14850,14859.20,255.80,102.10,153.70,167.30,NaN
7,2021-05-05,14600,14620.85,220.25,194.95,25.30,202.75,NaN
8,2021-05-11,14850,14850.40,254.20,296.00,-41.80,374.10,NaN
9,2021-06-02,15600,15587.45,261.00,206.40,54.60,340.25,205.40
